In [ ]:
#             Notebook #2
#                        Generate Part of Speech tags ad transition matrix 
#
#
#  Read (or reconstruct) the POS file containing all the POS for each sentence
#  Compute word frequencies
#  Construct the transition matrix and the histograms
#  Generate Honore and Brunete stats for:
#         Everything
#         Pronouns 
#
#  Standardized terms:
#       N: Total numbers of words emitted
#       V: Number of unique words (vocabulary)
#       V1: Number of words used once
#       P: Number of pronouns emitted
#       PV: Number of unique pronouns (pronoun vocabulary)
#       PV1: Number of pronouns ised once
#
#  Caveat Lector: 
#   This code runs on a specific directory structure. Yours will be different and so will require some
#   refactoring.

In [ ]:
#        Imports
import numpy as np
import pandas as pd
import stanza 
import tensorflow as tf
import tensorflow_hub as hub
from dateutil.parser import parse
import scipy as sp
from sklearn.preprocessing import scale
import numpy as np
import math
import sys,os
import gc
import phenograph 
#import networkx as nx
import igraph as ig
from sklearn import preprocessing
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import scipy.stats as stats
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import *
import matplotlib.dates as mdates
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.feature_selection import VarianceThreshold
import scipy as sp
import sklearn
from sklearn import linear_model
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn import feature_selection
from statsmodels.sandbox.stats.multicomp import multipletests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import FastICA, PCA
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import CategoricalNB
from sklearn import decomposition
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from docx.api import Document
import re
import string
from string import punctuation
from collections import Counter


#      Options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


In [ ]:
import pkg_resources
pkg_resources.get_distribution("stanza").version

In [ ]:
# Functions and Classes

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseQuotes": False}}})

#Figure out what variables are taking up memory
def varSizes():
    local_vars = list(locals().items())
    print(local_vars)
    varDF = pd.DataFrame(columns=['var','size'])
    for var, obj in local_vars:
        varDF.loc[len(varDF)]=[var, sys.getsizeof(obj)]
    varDF.sort_values(by=['size'],inplace=True,ascending=False)
    varDF.reset_index(inplace=True,drop=True)
    return varDF
    

def docToDF(doc):
    # Input: A Stanza doc object
    # Output: DataFrame object with one row for each token in

    #List of row data dictionaries
    rows = []
    for sentence in doc.sentences:
        for word in sentence.words:
            row = {
                "text": word.text,
                "lemma": word.lemma,
                "upos": word.upos,
                "xpos": word.xpos,
                "deprel": word.deprel,
            }
            rows.append(row)
    return pd.DataFrame(rows)
    
def parse_SessionSpeaker(transcriptFileName):
        #Full Transcript pathname version
        if transcriptFileName[0]=='/':
            path_file = os.path.split(transcriptFileName) 
            fileName = path_file[1]
        #LIWC filename version
        else:
            fileName = transcriptFileName
        
        fileName = fileName.split('.')[0]
        print('fileName: ', fileName)
        fileParts = fileName.split('_')

        Session = fileParts[0]
        print('Session : ', Session)
        
        Speaker = fileParts[1]
        print('Speaker : ', Speaker)
        
        return(Session,Speaker)

def ngramStats(allTextList,n):
    #Compute word frequencies
    word_fdist = nltk.FreqDist(allTextList)    
    wordDF = pd.DataFrame.from_dict(word_fdist, orient='index')
    wordDF['word'] = wordDF.index
    wordDF.columns = ['count','word']  
    wordDF = wordDF[['word','count']] 
    wordDF.sort_values(by=['count'], ascending=False, inplace=True)
    wordDF.reset_index(inplace=True,drop=True)
    wordDF.head()

    #Compute n gram frequencies
    ngram_fd = nltk.FreqDist(ngrams(allTextList,n))
    ngramDF = pd.DataFrame(ngram_fd.most_common(), columns =['w1-wn', 'Freq']) 
    return(wordDF,ngramDF)
 
def removeNonTerminatingPunctuation(allPOSDF):
    # Remove all non-terminating punctuation
    allPOSDF = allPOSDF.loc[allPOSDF['text']!=',',:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!=':',:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!=';',:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!='--',:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!='/',:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!='"',:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!='[',:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!=']',:] 
    allPOSDF = allPOSDF.loc[allPOSDF['text']!='-',:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!='(',:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!=')',:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!='“',:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!="'",:]
    allPOSDF = allPOSDF.loc[allPOSDF['text']!='”',:]
    return(allPOSDF)

#Populate the turnDF and posDF
def create_turnDF(all_text):
    turnDF = pd.DataFrame(columns=['turn','sentence','text'])
    turn = 0
    posDF = pd.DataFrame(columns=['sentence','text','lemma','upos','xpos','deprel'])
    sentenceTotalNum = 0
    all_text = '.'+all_text
    all_text = all_text.replace('\n','.\n')
    currentParts = all_text.split("\n")
    for i in range(len(currentParts)):
            text = currentParts[i]
            #                         Sentencize and Parse with stanza
            #Note once you get the text into a Stanza doc you can do all sorts of interesting things
            doc = nlp(text)

            sentenceNum = 0
            
            for sentence in doc.sentences:
               turnRow = [turn,sentenceNum,sentence.text]
               turnDF.loc[len(turnDF)+1] = turnRow
               
               #Populate the POS 
               for word in sentence.words:
                   posDF.loc[len(posDF)+1] = [sentenceTotalNum, word.text, word.lemma, word.upos, word.xpos, word.deprel]
               sentenceNum = sentenceNum + 1
               sentenceTotalNum = sentenceTotalNum + 1
               
            turn = turn + 1
    turnDF.reset_index(inplace=True,drop=True)     
    return [turnDF, posDF]


#           Calculate Honore's statistic
#Honore’s statistic [21] is based on the notion that the
#larger the number of words used by a speaker that occur
#only once, the richer his overall lexicon is. Words spoken
#only once (V1) and the total vocabulary used (V) have
#been shown to be linearly associated. Honore’s statistic
#generates a lexical richness measure according to R =
#100×log(N/(1−V1/V)), where N is the total text length.
#Higher values correspond to a richer vocabulary. As with
#standardized word entropy, stemming is done on words
#and only the stems are considered.
def honore(speakerDF):
    #Find words (drop punctuation)
    wordDF = speakerDF.loc[(speakerDF['upos']!='PUNCT'),:]
    wordList = list(wordDF['lemma'].dropna())
    numWords = len(wordList)
    wordCounter = Counter(wordList)
    wordFreq = wordCounter.most_common()
    #Display the word frequency
    oneTimeWords = []
    for word, count in wordCounter.most_common():
        #print(word,'\t', count*1.0/numWords*1.0)
        #print(word,'\t', count)
        if count==1:
            oneTimeWords.append(word)
    uniqueWordList = list(set(wordList))
    numUniqueWords = len(uniqueWordList)
    print('numUniqueWords: ', numUniqueWords)
    numOneTimeWords = len(oneTimeWords)
    print('numOneTimeWords: ',numOneTimeWords)
    D = 1.0-numOneTimeWords/numUniqueWords
    R = 100.0*(math.log(numWords/D,100))
    return(R)

#           Calculate Brunet’s index
#Brunet’s index (W) quantifies lexical richness [20]. It is
#calculated as W = NV**−0.165 , where N is the total text
#length and V is the total vocabulary. Lower values of W
#correspond to richer texts. As with standardized word
#entropy, stemming is done on words and only the stems
#are considered.
def brunet(speakerDF):
    #Find words (drop punctuation)
    wordDF = speakerDF.loc[(speakerDF['upos']!='PUNCT'),:]
    wordList = list(wordDF['lemma'].dropna())
    numWords = len(wordList)
    wordCounter = Counter(wordList)
    wordFreq = wordCounter.most_common()
    #Display the word frequency
    oneTimeWords = []
    for word, count in wordCounter.most_common():
        #print(word,'\t', count*1.0/numWords*1.0)
        #print(word,'\t', count)
        if count==1:
            oneTimeWords.append(word)
    uniqueWordList = list(set(wordList))
    numUniqueWords = len(uniqueWordList)
    W = (numWords*numUniqueWords)**-0.165
    return(W)

In [ ]:
#                                          Pull in models

stanza.download('en') # download English model
#Initialize a Stanza Pipeline
nlp = stanza.Pipeline(lang='en', processors='tokenize,lemma,pos,depparse,ner')

In [ ]:
"""                                    Penn Treebank Project POS Tags
Number
Tag
Description
1.	CC	Coordinating conjunction
2.	CD	Cardinal number
3.	DT	Determiner
4.	EX	Existential there
5.	FW	Foreign word
6.	IN	Preposition or subordinating conjunction
7.	JJ	Adjective
8.	JJR	Adjective, comparative
9.	JJS	Adjective, superlative
10.	LS	List item marker
11.	MD	Modal
12.	NN	Noun, singular or mass
13.	NNS	Noun, plural
14.	NNP	Proper noun, singular
15.	NNPS	Proper noun, plural
16.	PDT	Predeterminer
17.	POS	Possessive ending
18.	PRP	Personal pronoun
19.	PRP$	Possessive pronoun
20.	RB	Adverb
21.	RBR	Adverb, comparative
22.	RBS	Adverb, superlative
23.	RP	Particle
24.	SYM	Symbol
25.	TO	to
26.	UH	Interjection
27.	VB	Verb, base form
28.	VBD	Verb, past tense
29.	VBG	Verb, gerund or present participle
30.	VBN	Verb, past participle
31.	VBP	Verb, non-3rd person singular present
32.	VBZ	Verb, 3rd person singular present
33.	WDT	Wh-determiner
34.	WP	Wh-pronoun
35.	WP$	Possessive wh-pronoun
36.	WRB	Wh-adverb
"""

In [ ]:
#            Read transcripts for each video diary session and create a Pos file (if it doesn't already exist)
#
# Only need to do this once, unless you are changing something...
#
transcriptDir = '/Users/Heisig/Jihan/Transcripts/SplitTranscripts/'
posDir = '/Users/Heisig/Jihan/POS/'
resultsDir = '/Users/Heisig/Jihan/Results/'
liwcDyadFileName = '/Users/Heisig/Jihan/Results/liwcDyadData.csv'

In [ ]:
#                     allEmbeddingsDF processing
embDir = '/Users/Heisig/Jihan/Embeddings/'
#Read the embedding file...
allEmbeddingFileName = resultsDir+'AllEmbeddings.csv'

if os.path.isfile(allEmbeddingFileName):
   allEmbeddingsDF = pd.read_csv(allEmbeddingFileName)
else:
    #Build a list of all the embed files 
    emb_file_paths = []  # List which will store all of the full filepaths.
    # Walk the directory tree.
    for root, directories, files in os.walk(embDir):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            if 'Embedding.csv' in filepath:
                emb_file_paths.append(filepath)  # Add it to the list.
    emb_file_paths = sorted(emb_file_paths)
    print('emb_file_paths: \n',*list(emb_file_paths),sep='\n')

    allEmbeddingsDF = pd.DataFrame()

    for embFileName in emb_file_paths:
        #Read the Embedding file
        print('Reading: :',embFileName)
        embDF = pd.read_csv(embFileName) 
        embDF['sentence'] = embDF['sentence'] + sentenceCursor
        sentenceCursor = np.max(embDF['sentence'])+1
        allEmbeddingsDF = allEmbeddingsDF.append(embDF, ignore_index=True)

    allEmbeddingsDF.reset_index(inplace=True,drop=True)
    allEmbeddingsDF.to_csv(allEmbeddingFileName, index=False, header=True, sep=',')
    print('Saved: ',allEmbeddingFileName)

In [ ]:
allEmbeddingsDF.head()

In [ ]:
sorted(list(set(allEmbeddingsDF.subjectsession)))

In [ ]:
#             Read all the Parts of Speech files into a single data frame and save to a file (if it doesn't already exist...)

# Only need to do this once, unless you are changing something...
allPOSFileName = resultsDir+'AllPOS.csv'
sentenceCursor = 0

if os.path.isfile(allPOSFileName):
   allPOSDF = pd.read_csv(allPOSFileName)
   print('Reading: ',allPOSFileName)
else:
    #Build a list of all the pos files 
    pos_file_paths = []  # List which will store all of the full filepaths.
    # Walk the directory tree.
    for root, directories, files in os.walk(posDir):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            if '_POS' in filepath:
                pos_file_paths.append(filepath)  # Add it to the list.
    pos_file_paths = sorted(pos_file_paths)
    print('pos_file_paths: \n',*list(pos_file_paths),sep='\n')

    allPOSDF = pd.DataFrame()

    for posFileName in pos_file_paths:
        #Read the POS file
        print('Reading: :',posFileName)
        posDF = pd.read_csv(posFileName) 
        posDF['sentence'] = posDF['sentence'] + sentenceCursor
        sentenceCursor = np.max(posDF['sentence'])+1
        allPOSDF = allPOSDF.append(posDF, ignore_index=True)
        
    allPOSDF.reset_index(inplace=True,drop=True)
    allPOSDF.to_csv(allPOSFileName, index=False, header=True, sep=',')
    print('Saved: ',allPOSFileName)


In [ ]:
len(list(set(allPOSDF.subjectsession)))

In [ ]:
#                                    Build some indices

#Find the sentence numbers of all questions
punctAllDF = allPOSDF.groupby('sentence').last()
punctAllDF.reset_index(inplace=True)
questionsIdx = punctAllDF.loc[punctAllDF.text=='?','sentence']
#Find the sentence number of all NOT questions
notQuestionsIdx = punctAllDF.loc[punctAllDF.text!='?','sentence']

In [ ]:
#Find the set of ALL pronouns xpos version
xposPronDF = allPOSDF.loc[allPOSDF['xpos']=='PRP',:]
xposSet = list(set( xposPronDF['lemma'].str.lower()))
xposList = list(xposPronDF['lemma'].str.lower())

#Find the set of ALL pronouns upos version
uposAllPronDF = allPOSDF.loc[allPOSDF['upos']=='PRON',:]
uposAllPronounList = list(uposAllPronDF['lemma'].str.lower())
uposAllPronounList = [item for item in uposAllPronounList if item!='covid']
uposAllPronounSet = list(set(uposAllPronounList))
#Session Speaker
sortedPronounList = [item for items, c in Counter(uposAllPronounList).most_common() for item in [items] * c]
pronounFrequencyDict = Counter(sortedPronounList)
print(pronounFrequencyDict)

In [ ]:
commonPronounList = ['i', 'you', 'it', 'that', 'she', 'he', 'my', 'they', 'what', 'we', 'this', 'something', 'there', 'which', 'myself', 'who', 'anything', 'everything', 'yourself', 'whatever', 'somebody', 'us', 'someone', 'nothing', 'everybody', 'nobody', 'anybody', 'anyone', 'everyone', 'itself', 'mine', 'its', 'herself', 'himself', 'whose', 'themselves', 'yours', 'one', 'whoever', 'ourselves', 'her',
'his']

In [ ]:
#Compute ALL word frequency
wordDF = allPOSDF.loc[allPOSDF['upos']!='PUNCT',:]
words = list(wordDF['lemma'].dropna())
totalWords = len(words)
wordCounter = Counter(words)
wordFreq = wordCounter.most_common()
#Display the word frequency
for word, count in wordCounter.most_common(100):
    print(word,'\t', count*1.0/totalWords*1.0)

In [ ]:
#                                             Subset allPOSDF to a single session/speaker

#      Pick a session and speaker
#session = 31320  #Lowest Patient Alliance   
session = 101520  #Highest Patient Alliance 

speaker = 'therapist'
speakerDF = allPOSDF.loc[(allPOSDF['subjectsession']==session) & (allPOSDF['speaker']==speaker),:].copy()
speakerDF.reset_index(inplace=True,drop=True)
print(speakerDF.head())
speakerDF['tokenNum'] = speakerDF.index
#Realign the sentence number
speakerDF['speakerSentenceNum'] = speakerDF['sentence']
speakerDF['speakerSentenceNum'] = speakerDF['speakerSentenceNum'] - speakerDF.loc[0,'speakerSentenceNum']
print(speakerDF.shape)

uposPronDF = speakerDF.loc[(speakerDF['upos']=='PRON'),:]
uposPronounList = list(uposPronDF['lemma'].str.lower())
uposPronounList = [item for item in uposPronounList if item!='covid']
uposPronounSet = list(set(uposPronounList))
sortedPronounList = [item for items, c in Counter(uposPronounList).most_common() for item in [items] * c]


uposIntjDF = speakerDF.loc[(speakerDF['upos']=='INTJ'),:]
uposIntjList = list(uposIntjDF['lemma'].str.lower())
uposIntjList = [item for item in uposIntjList if item!='covid']
uposIntjSet = list(set(uposIntjList))
sortedIntjList = [item for items, c in Counter(uposIntjList).most_common() for item in [items] * c]


In [ ]:
allPOSDF.head()

In [ ]:
#                                                   Plot Pronouns
sns.set_style('darkgrid')
fig, ax = plt.subplots()
fig.set_size_inches(16,16)

#Plot Histogram
sns.histplot(sortedPronounList, bins=len(sortedPronounList), label='Pronouns ' , color='limegreen')

#Title and Axis labels  
ax.set_title('  Distribution of Pronouns '+str(session)+' '+speaker,fontsize='xx-large')    

plt.legend(fontsize='x-large', title_fontsize='40')
plt.setp(ax.get_xticklabels(), rotation=30, fontsize='medium');

plt.xlabel('Pronoun',fontsize='x-large', weight='bold')
plt.ylabel('Observations',fontsize='x-large', weight='bold')

plt.show()

In [ ]:
#Session Speaker
pronounFrequencyDict = Counter(sortedPronounList)
print(pronounFrequencyDict)

intjFrequencyDict = Counter(sortedIntjList)
print(intjFrequencyDict)

In [ ]:
#Compute the session speaker word frequency
wordDF = speakerDF.loc[speakerDF['upos']!='PUNCT',:]
words = list(wordDF['lemma'].dropna())
totalWords = len(words)
print(totalWords,' totalwords')
wordCounter = Counter(words)
wordFreq = wordCounter.most_common()
#Display the word frequency
for word, count in wordCounter.most_common(100):
    print(word,'\t', count*1.0/totalWords*1.0)

In [ ]:
count/totalWords

In [ ]:
#Compute the upos token frequency
tokens = list(speakerDF['upos'].dropna())
tokenCounter = Counter(tokens)
tokenFreq = tokenCounter.most_common()
#Display the word frequency
for token, count in tokenCounter.most_common(100):
    print('%s %7d' % (token, count))

In [ ]:
allPOSDF.columns

In [ ]:
#                                                   Plot UPOS tokens
sns.set_style('darkgrid')
fig, ax = plt.subplots()
fig.set_size_inches(16,16)

transitions = speakerDF['upos']
tokenCounter = Counter(transitions)
uposTokens = tokenCounter.most_common()

uposTokens = [item for items, c in Counter(transitions).most_common() for item in [items] * c]

#Plot Histogram
sns.histplot(uposTokens, bins=17, label='UPOS ' , color='limegreen')

#Title and Axis labels  
ax.set_title('  Distribution of UPOS tokens '+str(session)+' '+speaker,fontsize='xx-large')    

plt.legend(fontsize='x-large', title_fontsize='40')
#tickPositions = list(range(10))
#tickLabels = list(range(10))
#plt.xticks(tickPositions,tickLabels,fontsize=35)
plt.setp(ax.get_xticklabels(), rotation=30, fontsize='medium');

plt.xlabel('UPOS',fontsize='x-large', weight='bold')
plt.ylabel('Observations',fontsize='x-large', weight='bold')

plt.show()

In [ ]:
#Compute the xpos token frequency
tokens = list(speakerDF['xpos'].dropna())
xposList = list(set(tokens))
tokenCounter = Counter(tokens)
tokenFreq = tokenCounter.most_common()
#Display the word frequency
for token, count in tokenCounter.most_common(100):
    print('%s %7d' % (token, count))

In [ ]:
#                                                   Plot XPOS tokens
sns.set_style('darkgrid')
fig, ax = plt.subplots()
fig.set_size_inches(16,16)

transitions = speakerDF['xpos']
tokenCounter = Counter(transitions)
uposTokens = tokenCounter.most_common()

uposTokens = [item for items, c in Counter(transitions).most_common() for item in [items] * c]

#Plot Histogram
sns.histplot(uposTokens, bins=len(xposList), label='XPOS ' , color='limegreen')

#Title and Axis labels  
ax.set_title('  Distribution of XPOS tokens '+str(session)+' '+speaker,fontsize='xx-large')    

plt.legend(fontsize='x-large', title_fontsize='40')
#tickPositions = list(range(10))
#tickLabels = list(range(10))
#plt.xticks(tickPositions,tickLabels,fontsize=35)
plt.setp(ax.get_xticklabels(), rotation=30, fontsize='medium');

plt.xlabel('XPOS',fontsize='x-large', weight='bold')
plt.ylabel('Observations',fontsize='x-large', weight='bold')

plt.show()

In [ ]:
#         Compute and Plot the UPOS Transition matrix

In [ ]:
#The set of upos tokens (states)
uposList = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
uposNum = len(uposList)
len(uposList)

In [ ]:
#                        Function to calculate the transition matrix and transition vector for all speakerDFs

def calcTranDF(speakerDF):
    #Compute the transition matrix for the upos tokens
    def calcRowProb(row):
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
        return row

    #The set of upos tokens (states)
    uposList = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
    uposNum = len(uposList)
    
    #Define the transition matrix
    transDF = pd.DataFrame(np.zeros((uposNum,uposNum)))

    #Define the element to index dictionary
    allUposDict = {k: v for v, k in enumerate(uposList)}
    #print(allUposDict)

    idDF = pd.DataFrame(np.zeros((uposNum,uposNum)))
    
    #Get rid of internal sentence punctuation
    cleanPOSDF = removeNonTerminatingPunctuation(speakerDF)
    transitionChars = cleanPOSDF['upos']

    #Apply the dictionary to switch from upos tokens to indices
    transitions = [allUposDict[k] for k in transitionChars]

    #Count the transitions (this needs to be a list so we can look at it later, a zip obj gets 'consumed')
    transitionTupleList = list(zip(transitions,transitions[1:]))
    transitionCharTupleList = list(zip(transitionChars,transitionChars[1:]))
    print('transitionCharTupleList: \n',transitionCharTupleList)
    for (i,j) in transitionTupleList:
            transDF.iloc[i,j] += 1

    #Convert the counts to probabilities:        
    transDF.apply(calcRowProb, axis=1)

    #Add column and row names
    transDF.columns = uposList
    transDF.index = uposList
    
    idDF = pd.DataFrame(columns=uposList,index=uposList)
    for c in uposList:
        for r in uposList:
            idDF.at[r,c] = r+','+c

    #Extract the transition vector from the triangular matrix (including the diagonal!!!)
    rows,cols = transDF.shape
    transNP = transDF.to_numpy()
    transVector = transNP[np.triu_indices(rows, k = 0)]
    transVector = pd.Series(transVector)
    
    #Figure out the names and label the transition vector
    idDF.columns=range(uposNum)
    idDF.index=range(uposNum)
    indices = np.triu_indices(uposNum, k = 0)
    tupleList = list(zip(indices[0],indices[1]))
    vectorIdList = [] 
    for t in tupleList:
        vectorIdList.append(idDF.at[t]) 
    transVector.index = vectorIdList
    
    return(transDF,transVector)

In [ ]:

#                               Calculate and save the pronoun vectors for all the speaker/session pairs

# Values are:  Frequency I/You Self/Other, frequency of each pronoun wrt all speech


commonPronounList = ['i', 'you', 'it', 'that', 'she', 'he', 'my', 'they', 'what', 'we', 'this', 'something', 'there', 'which', 'myself', 'who', 'anything', 'everything', 'yourself', 'whatever', 'somebody', 'us', 'someone', 'nothing', 'everybody', 'nobody', 'anybody', 'anyone', 'everyone', 'itself', 'mine', 'its', 'herself', 'himself', 'whose', 'themselves', 'yours', 'one', 'whoever', 'ourselves', 'her',
'his'] 
pronounsPrefixedList =  ['pronoun_' + pronoun for pronoun in commonPronounList]
allSessions = list(set(allPOSDF['subjectsession']))
speakers = ['therapist','patient']
transDir = '/Users/Heisig/Jihan/POS_Transitions/'
resultsDir = '/Users/Heisig/Jihan/Results/'
vectorList = []
sampleList = []

columnList = ['Session', 'Speaker', 'pronoun_Honore', 'pronoun_Brunet', 'pronoun_IY', 'pronoun_SO', 'pronoun_TP', 'pronoun_WE']+commonPronounList
finalColumnList = ['Session', 'Speaker', 'pronoun_Honore', 'pronoun_Brunet', 'pronoun_IY', 'pronoun_SO', 'pronoun_TP', 'pronoun_WE']+pronounsPrefixedList
allPronounDF = pd.DataFrame(columns = columnList)

for session in allSessions:
    for speaker in speakers:
        print(session,speaker)
        sampleList.append(str(session)+'-'+speaker)
        speakerDF = allPOSDF.loc[(allPOSDF['subjectsession']==session) & (allPOSDF['speaker']==speaker),:].copy()
        speakerDF.reset_index(inplace=True,drop=True)
        speakerDF['tokenNum'] = speakerDF.index
        
        
        #Find the set of ALL pronouns upos version
        uposAllPronDF = speakerDF.loc[speakerDF['upos']=='PRON',:]
        uposAllPronounList = list(uposAllPronDF['lemma'].str.lower())
        uposAllPronounList = [item for item in uposAllPronounList if item!='covid']
        uposAllPronounSet = list(set(uposAllPronounList))
        #Session Speaker
        sortedPronounList = [item for items, c in Counter(uposAllPronounList).most_common() for item in [items] * c]
        pronounFrequencyDict = Counter(sortedPronounList)
        #print(pronounFrequencyDict)
        
        #Calculate Honore's statistic
        R = honore(uposAllPronDF)
        print("Honore's statistic: ",R)
        
        #Calculate Honore's statistic
        W = brunet(uposAllPronDF)
        print("Brunet's Index: ",W)
        
        #Compute pronoun frequencies
        rows,cols = speakerDF.shape
        pronounVec = [0]*len(commonPronounList)
        idx = 0
        for pronoun in commonPronounList:
            if pronoun in pronounFrequencyDict:
                count = pronounFrequencyDict[pronoun]/float(rows)
                pronounVec[idx] = count
            idx = idx + 1

        #Create the rowDF
        #Init I/You, Self vs Other and Third Party
        IY = 0
        SO = 0
        TP = 0
        WE = 0
        pronounRow = [session, speaker, R, W, IY, SO, TP, WE]+pronounVec
        pronounRowDF = pd.DataFrame([pronounRow],columns=columnList)
        
        #Compute I/You and Self vs Other
        pronounRowDF['pronoun_IY'] =  pronounRowDF['i']/pronounRowDF['you']
        pronounRowDF['pronoun_SO'] = (pronounRowDF['i']+pronounRowDF['my']+pronounRowDF['myself'])/(pronounRowDF['you']+pronounRowDF['yours']+pronounRowDF['yourself'])
        pronounRowDF['pronoun_TP'] =  pronounRowDF['she']+pronounRowDF['he']+pronounRowDF['they']+pronounRowDF['somebody']+pronounRowDF['someone']+pronounRowDF['everybody']+pronounRowDF['nobody']+pronounRowDF['anybody']+pronounRowDF['anyone']+pronounRowDF['everyone']
        pronounRowDF['pronoun_WE'] =  pronounRowDF['we']+pronounRowDF['us']+pronounRowDF['ourselves']
        #Mash this person's pronouns into the common DF  
        allPronounDF = allPronounDF.append(pronounRowDF, ignore_index=True)  

allPronounDF.columns = finalColumnList
allPronounFileName = resultsDir + 'allPronounDF.csv' 
allPronounDF.to_csv(allPronounFileName, index=False, header=True, sep=',')

In [ ]:
allPronounDF

In [ ]:

#                   Calculate the transition matrices and vectors for all the speaker/session pairs

allSessions = list(set(allPOSDF['subjectsession']))
speakers = ['therapist','patient']
transDir = '/Users/Heisig/Jihan/POS_Transitions/'
resultsDir = '/Users/Heisig/Jihan/Results/'

allTranDF = pd.DataFrame()

for session in allSessions:
    for speaker in speakers:
        #print(session,speaker)
        speakerDF = allPOSDF.loc[(allPOSDF['subjectsession']==session) & (allPOSDF['speaker']==speaker),:].copy()
        speakerDF.reset_index(inplace=True,drop=True)
        speakerDF['tokenNum'] = speakerDF.index
        #Realign the sentence number
        speakerDF['speakerSentenceNum'] = speakerDF['sentence']
        speakerDF['speakerSentenceNum'] = speakerDF['speakerSentenceNum'] - speakerDF.loc[0,'speakerSentenceNum']
        #print(speakerDF.shape)
        [transDF,transVec] = calcTranDF(speakerDF)
        #Save the individual session/speaker transition matrix
        transFileName = transDir + str(session) + '_' + speaker + '_tranDF.csv' 
        transDF.to_csv(transFileName, index=False, header=True, sep=',')
        
        tranRow = [session, speaker]
        tranRow.extend(transVec.tolist())
        
        columnList = ['Session', 'Speaker']
        columnList.extend(transVec.index.tolist())
        
        tranRowDF = pd.DataFrame([tranRow],columns=columnList)
        allTranDF = allTranDF.append(tranRowDF, ignore_index=True)

allTranFileName = resultsDir + 'allPOStransDF.csv' 
allTranDF.to_csv(allTranFileName, index=False, header=True, sep=',')
allTranDF.head()

In [ ]:

#                   Calculate the Honore and Brunete for all words for each speaker/session pair

allSessions = list(set(allPOSDF['subjectsession']))
#speakers = ['therapist','patient']
speakers = ['therapist','patient']
resultsDir = '/Users/Heisig/Jihan/Results/'
vectorList = []
for session in allSessions:
    for speaker in speakers:
        #print(session,speaker)
        speakerDF = allPOSDF.loc[(allPOSDF['subjectsession']==session) & (allPOSDF['speaker']==speaker),:].copy()
        speakerDF.reset_index(inplace=True,drop=True)
        speakerDF['tokenNum'] = speakerDF.index
        #Realign the sentence number
        speakerDF['speakerSentenceNum'] = speakerDF['sentence']
        speakerDF['speakerSentenceNum'] = speakerDF['speakerSentenceNum'] - speakerDF.loc[0,'speakerSentenceNum']
        #print(speakerDF.shape)
        
        #Calculate Honore's statistic
        R = honore(speakerDF)
        #print("Honore's statistic: ",R)
        W = brunet(speakerDF)
        #print("Brunet's Index: ",W)
        vectorList.append([session,speaker,R,W])
        
hbDF = pd.DataFrame(vectorList)
hbDF.columns=['Session','Speaker','Honore','Brunet']

hbFileName = resultsDir + 'honoreBrunetDF.csv' 
hbDF.to_csv(hbFileName, index=False, header=True, sep=',')
hbDF.head()

In [ ]:
# Frequency of transitions
transitionCharTupleList[0:25]
#Compute the upos token frequency
tokens = transitionCharTupleList
tokenCounter = Counter(tokens)
tokenFreq = tokenCounter.most_common()
#Display the word frequency
for token, count in tokenCounter.most_common(100):
    print('%s %7d' % (token, count))

In [ ]:
transitionTupleList[0:25]

In [ ]:
transDF

In [ ]:
#Plot the transition matrix
plt.figure(figsize=(22,22))
sns.heatmap(transDF, annot=False, cmap='coolwarm', xticklabels=uposList, yticklabels=uposList)
plt.tick_params(axis='both', which='major', labelsize=15, labelbottom = False, bottom=False, top = False, labeltop=True)
plt.show()

In [ ]:
rows,cols = transDF.shape
print(transDF.shape)
print('total elements: ',rows*cols)
transNP = transDF.to_numpy()
transVector = transNP[np.triu_indices(rows, k = 1)]
print('transVector length: ',len(transVector))
print(transVector)

In [ ]:
#                      Evaluate a sentence using the transition matrix
#  First: Sentencize and Parse with stanza

rows = []

#SVO example
example_text = ".Let's stick to the plan."

doc = nlp(example_text)

sentenceTotalNum = 0
for sentence in doc.sentences:
   for word in sentence.words:
            row = {
                "sentence": sentenceTotalNum,
                "text": word.text,
                "lemma": word.lemma,
                "upos": word.upos,
                "xpos": word.xpos,
                "deprel": word.deprel,
            }
            rows.append(row)
   sentenceTotalNum = sentenceTotalNum + 1     

sentenceDF = pd.DataFrame(rows) 
sentenceDF

In [ ]:
#                                       Evaluate the sentence with the transition matrix
# Basically run the upos sequence
sentenceTokens = list(sentenceDF['upos'])
print('sentenceText: \n',sentenceDF['text'])
print('sentenceTokens: ',sentenceTokens)
sentenceSequenceTupleList = list(zip(sentenceTokens,sentenceTokens[1:]))
#print('sentenceSequenceTupleList: ',sentenceSequenceTupleList)

# Compute the perplexity for the test sentence given the 805 transition matrix
# token transition probablity
ttp = 1.0
N = len(sentenceSequenceTupleList)
print('N: ',N)
for (i,j) in sentenceSequenceTupleList:
        print(transDF[i][j])
        ttp = ttp*transDF[i][j]
print('total transition probability: ',ttp)

#Normalize for sentence length
ppx = (1.0/ttp)**(1.0/N)
print('perplexity: ',ppx)

In [ ]:
#   Find sentences with a specific words
targetLemma = 'I'
#Find sentence numbers containing the target word
targetDF = speakerDF.loc[(speakerDF['lemma']==targetLemma),:]
targetDF.head(n=100)

In [ ]:
#   Find sentences with various combinations of words and POS
#Find sentence numbers containing the first target word
speakerDF = allPOSDF.loc[allPOSDF['speaker']=='Interviewer',:].copy()
targetLemma = 'I'
targetDF = speakerDF.loc[(speakerDF['lemma']==targetLemma)&(speakerDF['deprel']=='nsubj'),:]
targetSentenceNumberList = sorted(list(set(targetDF['sentence'])))
#Show the list of sentences containing the target word in its specified role
targetSentencesDF = speakerDF.loc[speakerDF['sentence'].isin(targetSentenceNumberList),:] 
print(targetSentencesDF)

#Find sentence numbers containing the second target word
targetLemma = 'you'
targetDF = targetSentencesDF.loc[(targetSentencesDF['lemma']==targetLemma)&(targetSentencesDF['deprel']=='obj'),:]
targetSentenceNumberList = sorted(list(set(targetDF['sentence'])))
#Show the list of sentences containing the target word in its specified role
targetSentencesDF = speakerDF.loc[speakerDF['sentence'].isin(targetSentenceNumberList),:] 

In [ ]:
targetDF.head()

In [ ]:
speakerDF = allPOSDF.loc[allPOSDF['speaker']=='Interviewer',:].copy()
targetLemma = 'so'
targetDF = speakerDF.loc[(speakerDF['lemma']==targetLemma),:]
targetSentenceNumberList = sorted(list(set(targetDF['sentence'])))
#Show the list of sentences containing the target word in its specified role
targetSentencesDF = speakerDF.loc[speakerDF['sentence'].isin(targetSentenceNumberList),:] 

In [ ]:
#targetSentencesDF
#speakerDF[0:100]
targetSentencesDF

In [ ]:
#Define the plot
fig, ax = plt.subplots()
fig.set_size_inches(24, 8)

ax = sns.scatterplot(data=targetDF,
                     x='tokenNum',
                     y='speakerSentenceNum',
                     palette="deep",
                     hue='deprel',
                     s=50,
            # linewidth in a scatterplot removes the white circle
                     linewidth=0,
                     alpha=1.0)

plt.xlabel('token position',fontsize='large', weight='bold')
plt.ylabel('sentenceNum',fontsize='large', weight='bold')
plt.show()



In [ ]:
#Recover all the sentence strings containing the target word
targetSentenceList = []
for s in targetSentenceNumberList:
    sentenceDF = targetSentencesDF.loc[targetSentencesDF['sentence']==s]
    sentence = ' '.join(sentenceDF['text'])
    targetSentenceList.append(sentence)
targetSentenceList    

In [ ]:
#Pull out instances of a POS transition
#pos1 = 'AUX'
#pos2 = 'INTJ'

#pos1 = 'CCONJ'
#pos2 = 'X'

#pos1 = 'PRON'
#pos2 = 'PRON'

#pos1 = 'ADP'
#pos2 = 'INTJ'


pos1 = 'SCONJ'
pos2 = 'PRON'

posTranDF = pd.DataFrame(columns=['session','speaker','sentenceNum','POS1','POS2','sentenceText']) 
pos1DF = allPOSDF.loc[allPOSDF['upos']==pos1,:].copy()
for idx in pos1DF.index:
    if allPOSDF.loc[idx+1,'upos']==pos2:
        sentenceNum = pos1DF.loc[idx,'sentence']
        sentenceDF = allEmbeddingsDF.loc[allEmbeddingsDF['sentence']==sentenceNum,'text']
        sentenceDF.reset_index(inplace=True,drop=True)
        sentenceText = sentenceDF[0]
        newRow = [pos1DF.loc[idx,'subjectsession'],pos1DF.loc[idx,'speaker'],sentenceNum,pos1DF.loc[idx,'text'],allPOSDF.loc[idx+1,'text'],sentenceText]
        print('New row: ',newRow)
        posTranDF.loc[len(posTranDF)] = newRow
#targetDF = speakerDF.loc[(speakerDF['lemma']==targetLemma),:]
#targetSentenceNumberList = sorted(list(set(targetDF['sentence'])))
#Show the list of sentences containing the target word in its specified role
#targetSentencesDF = speakerDF.loc[speakerDF['sentence'].isin(targetSentenceNumberList),:] 
posTranFileName = resultsDir + pos1+'_'+pos2+'_POS_transDF.csv' 
posTranDF.to_csv(posTranFileName, index=False, header=True, sep=',')
posTranDF

In [ ]:
sentenceDF

In [ ]:
for index, row in posTranDF.iterrows():
    sentence = row['sentence']
    sentenceDF = allEmbeddingsDF.loc[allEmbeddingsDF['sentence']==sentence,'text']
    sentenceDF.reset_index(inplace=True,drop=True)
    sentenceText = sentenceDF[0]
    print(sentenceText)

In [ ]:
len(list(set(allPOSDF.subjectsession)))

In [ ]:
pos1DF.index

In [ ]:
#Recover all the n-grams containing the target word
totalNgramDF = pd.DataFrame()
for tokenNum in targetDF['tokenNum']:
    ngramDF = speakerDF.iloc[[tokenNum-1,tokenNum,tokenNum+1],:]
    totalNgramDF = totalNgramDF.append(ngramDF, ignore_index=True)
totalNgramDF

In [ ]:
speakerDF.iloc[[2,3,4],:]

In [ ]:
allPOSDF.head()

In [ ]:
#                                 Loop through all the sessions and build the allPosFeaturesDF dataframe
allSessions = list(set(allPOSDF['subjectsession']))
speakers = ['therapist','patient']

allPosFeaturesDF = pd.DataFrame(columns=['Session','Speaker','ISYO_count','ISYO_list'])

for session in allSessions:
    for speaker in speakers:
        speakerDF = allPOSDF.loc[(allPOSDF['subjectsession']==session) & (allPOSDF['speaker']==speaker),:].copy()
        print('session,speaker: ',session,speaker)
        print('speakerDF.shape: ',speakerDF.shape)
        speakerDF.reset_index(inplace=True,drop=True)
        speakerDF['tokenNum'] = speakerDF.index
        #Realign the sentence number
        speakerDF['speakerSentenceNum'] = speakerDF['sentence']
        speakerDF['speakerSentenceNum'] = speakerDF['speakerSentenceNum'] - speakerDF.loc[0,'speakerSentenceNum']
        print(speakerDF.shape)

        uposPronDF = speakerDF.loc[(speakerDF['upos']=='PRON'),:]
        uposPronounList = list(uposPronDF['lemma'].str.lower())
        uposPronounList = [item for item in uposPronounList if item!='covid']
        uposPronounSet = list(set(uposPronounList))
        sortedPronounList = [item for items, c in Counter(uposPronounList).most_common() for item in [items] * c]

        uposIntjDF = speakerDF.loc[(speakerDF['upos']=='INTJ'),:]
        uposIntjList = list(uposIntjDF['lemma'].str.lower())
        uposIntjList = [item for item in uposIntjList if item!='covid']
        uposIntjSet = list(set(uposIntjList))
        sortedIntjList = [item for items, c in Counter(uposIntjList).most_common() for item in [items] * c]

        
        #   Find sentences with various combinations of words
        #Find sentence numbers containing the first target word
        targetLemma = 'I'
        targetDF = speakerDF.loc[(speakerDF['lemma']==targetLemma)&(speakerDF['deprel']=='nsubj'),:]
        targetSentenceNumberList = sorted(list(set(targetDF['sentence'])))
        #Show the list of sentences containing the target word in its specified role
        targetSentencesDF = speakerDF.loc[speakerDF['sentence'].isin(targetSentenceNumberList),:] 

        #Find sentence numbers containing the second target word
        targetLemma = 'you'
        targetDF = targetSentencesDF.loc[(targetSentencesDF['lemma']==targetLemma)&(targetSentencesDF['deprel']=='obj'),:]
        targetSentenceNumberList = sorted(list(set(targetDF['sentence'])))
        #Show the list of sentences containing the target word in its specified role
        targetSentencesDF = speakerDF.loc[speakerDF['sentence'].isin(targetSentenceNumberList),:] 
        newRow = [session, speaker, len(targetSentenceNumberList),targetSentenceNumberList]
        allPosFeaturesDF.loc[len(allPosFeaturesDF)] = newRow                 
                  


In [ ]:
allPosTherapistFeaturesDF = allPosFeaturesDF.loc[allPosFeaturesDF['Speaker']=='therapist'].copy()

In [ ]:
allPosTherapistFeaturesDF.sort_values(by=['ISYO_count'],inplace=True,ascending=False)
allPosTherapistFeaturesDF

In [ ]:
for index, row in allPosTherapistFeaturesDF.iterrows():
  print(row['Session'])
  isyo_sentenceList = row['ISYO_list']
  for sentence in isyo_sentenceList:
    sentenceDF = allEmbeddingsDF.loc[allEmbeddingsDF['sentence']==sentence,'text']
    sentenceDF.reset_index(inplace=True,drop=True)
    sentenceText = sentenceDF[0]
    print(sentenceText)